In [ ]:
import pandas as pd
from ipywidgets import FileUpload, Button, VBox, Label
from IPython.display import display
from io import StringIO
from difflib import SequenceMatcher

In [ ]:
# Interface
label1 = Label("📄 Selecione a Planilha 1 (antiga):")
upload1 = FileUpload(accept='.csv', multiple=False)

label2 = Label("📄 Selecione a Planilha 2 (nova):")
upload2 = FileUpload(accept='.csv', multiple=False)

button = Button(description="🔄 Gerar Planilhas", button_style='success')
output_label = Label()


def escolher_valor(antigo, novo, coluna):
    """Mescla inteligente entre valores antigo e novo, com detecção de mudança de pessoa."""
    antigo = str(antigo).strip() if pd.notna(antigo) else ""
    novo = str(novo).strip() if pd.notna(novo) else ""

    # Se ambos vazios, nada a fazer
    if not antigo and not novo:
        return ""

    # 🔍 Se for nome, avaliar similaridade
    if "nome" in coluna.lower() and antigo and novo:
        similaridade = SequenceMatcher(None, antigo.lower(), novo.lower()).ratio()

        # Se nomes são completamente diferentes → prioriza o novo (mudança de pessoa)
        if similaridade < 0.5:
            return novo

        # Se são parecidos → mantém o mais completo
        return novo if len(novo) >= len(antigo) else antigo

    # Se novo está vazio, mantém o antigo
    if not novo:
        return antigo

    # Se novo é diferente, usa o novo
    return novo


def process_files(_):
    if not upload1.value or not upload2.value:
        output_label.value = "⚠️ Selecione as duas planilhas antes de continuar!"
        return

    # Ler CSVs
    file1 = list(upload1.value.values())[0]
    file2 = list(upload2.value.values())[0]
    df1 = pd.read_csv(StringIO(file1['content'].decode('utf-8')))
    df2 = pd.read_csv(StringIO(file2['content'].decode('utf-8')))

    # Verificar colunas
    if not all(df1.columns == df2.columns):
        output_label.value = "❌ As planilhas devem ter as mesmas colunas!"
        return

    # Identificar coluna de telefone
    possible_keys = [col for col in df1.columns if "fone" in col.lower() or "tel" in col.lower()]
    if not possible_keys:
        output_label.value = "❌ Nenhuma coluna de telefone encontrada! Renomeie uma coluna para 'Telefone'."
        return

    telefone_col = possible_keys[0]

    # Padronizar telefones
    df1[telefone_col] = df1[telefone_col].astype(str).str.replace(r'\D', '', regex=True)
    df2[telefone_col] = df2[telefone_col].astype(str).str.replace(r'\D', '', regex=True)

    # Remover duplicados
    df1 = df1.drop_duplicates(subset=[telefone_col], keep='last')
    df2 = df2.drop_duplicates(subset=[telefone_col], keep='last')

    # Mesclar bases
    merged_df = pd.merge(df1, df2, on=telefone_col, how='outer', suffixes=('_antigo', '_novo'))
    base_cols = [col for col in df1.columns if col != telefone_col]

    final_data = []
    alterados = []

    for _, row in merged_df.iterrows():
        registro_final = {telefone_col: row[telefone_col]}
        registro_antigo = {col: row.get(f"{col}_antigo", "") for col in base_cols}
        houve_alteracao = False

        # Preparar observação antiga
        observacao_final = str(row.get("observação_antigo", "")).strip() if "observação" in df1.columns else ""

        for col in base_cols:
            valor_antigo = row.get(f"{col}_antigo", "")
            valor_novo = row.get(f"{col}_novo", "")
            valor_final = escolher_valor(valor_antigo, valor_novo, col)

            # Caso especial: mudança de origem
            if col.lower() == "origem":
                origem_antiga = str(valor_antigo).strip()
                origem_nova = str(valor_novo).strip()
                if origem_antiga and origem_nova and origem_antiga != origem_nova:
                    # adicionar ao final da observação
                    complemento = f"entrada anterior: {origem_antiga}"
                    if observacao_final:
                        observacao_final += ". " + complemento
                    else:
                        observacao_final = complemento

            registro_final[col] = valor_final

            # Detectar mudança real
            antigo = str(valor_antigo).strip()
            if valor_final != antigo and antigo:
                houve_alteracao = True

        # Atualiza observação (depois de processar tudo)
        if "observação" in df1.columns:
            registro_final["observação"] = observacao_final

        final_data.append(registro_final)

        if houve_alteracao:
            alterados.append({
                telefone_col: row[telefone_col],
                **registro_antigo,
                **{f"{col}_final": registro_final[col] for col in base_cols}
            })

    # Criar DataFrames
    final_df = pd.DataFrame(final_data)
    alterados_df = pd.DataFrame(alterados)

    # Gerar relatório
    total_antiga = len(df1)
    total_nova = len(df2)
    total_final = len(final_df)
    novos_contatos = len(df2[~df2[telefone_col].isin(df1[telefone_col])])
    contatos_atualizados = len(alterados_df)

    relatorio = f"""
📊 RELATÓRIO DE MUDANÇAS (Mesclagem Inteligente)
───────────────────────────────────────────────
🔹 Registros na planilha antiga: {total_antiga}
🔹 Registros na planilha nova: {total_nova}
🔹 Total na planilha final: {total_final}

🆕 Novos contatos adicionados: {novos_contatos}
🔁 Contatos atualizados (dados mesclados): {contatos_atualizados}

📄 Arquivos gerados:
- planilha_final.csv
- planilha_alterados.csv
"""

    # Salvar arquivos
    final_df.to_csv("planilha_final.csv", index=False)
    alterados_df.to_csv("planilha_alterados.csv", index=False)
    with open("relatorio_mudancas.txt", "w", encoding="utf-8") as f:
        f.write(relatorio)

    output_label.value = (
        "✅ Arquivos gerados com sucesso!\n"
        "📁 planilha_final.csv — base consolidada\n"
        "📁 planilha_alterados.csv — somente contatos com mudanças\n"
        "📄 relatorio_mudancas.txt — resumo estatístico"
    )


# Conectar botão e exibir interface
button.on_click(process_files)
display(VBox([label1, upload1, label2, upload2, button, output_label]))
